In [1]:
%pip install --quiet -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Setup Ambiente

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from matplotlib.ticker import ScalarFormatter
import gc
import time
import tracemalloc
import os

from algoritmo_prim.arvore_geradora_minima import prim
from algoritmo_prim.arvore_geradora_minimo_exibicao import prim_interativo_teclado,salvar_grafo_agm_pronto

# Função Gerar Grafo

In [3]:
def gerar_matriz_ponderada_aleatoria(linhas, colunas, peso_min=1, peso_max=10):
    """
    Gera uma matriz de adjacência simétrica (grafo não direcionado) com pesos aleatórios.
    - linhas, colunas: número de vértices (a matriz será quadrada, usa o maior dos dois)
    - peso_min, peso_max: intervalo dos pesos das arestas
    """
    n = max(linhas, colunas)
    matriz = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            peso = random.randint(peso_min, peso_max)
            matriz[i][j] = peso
            matriz[j][i] = peso
    return matriz

In [4]:
matriz = gerar_matriz_ponderada_aleatoria(10, 10, 1, 10)

In [5]:
matriz

[[0, 1, 6, 2, 6, 2, 5, 9, 4, 9],
 [1, 0, 6, 9, 4, 3, 3, 4, 2, 2],
 [6, 6, 0, 9, 10, 10, 6, 2, 1, 9],
 [2, 9, 9, 0, 5, 9, 5, 7, 2, 10],
 [6, 4, 10, 5, 0, 8, 9, 10, 3, 8],
 [2, 3, 10, 9, 8, 0, 3, 6, 4, 2],
 [5, 3, 6, 5, 9, 3, 0, 5, 4, 4],
 [9, 4, 2, 7, 10, 6, 5, 0, 1, 9],
 [4, 2, 1, 2, 3, 4, 4, 1, 0, 4],
 [9, 2, 9, 10, 8, 2, 4, 9, 4, 0]]

# Função para teste

In [6]:
def measure_performance(func, args):
    gc.collect()  # Força coleta de lixo antes da medição
    
    # Mede tempo
    inicio = time.time()
    
    # Mede memória
    tracemalloc.start()
    result = func(*args)  # Desempacota os argumentos do tuple
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    # Calcula tempo
    fim = time.time()
    tempo = fim - inicio
    
    return result, tempo, peak / 1024

# Gerar Grafos

In [7]:
teste = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150]

In [8]:
grafos_gerados = [gerar_matriz_ponderada_aleatoria(teste[i], teste[i]) for i in range(len(teste))]

In [9]:
grafos_gerados

[[[0, 2, 9, 7, 9, 2, 7, 1, 5, 8],
  [2, 0, 2, 5, 5, 3, 5, 6, 10, 1],
  [9, 2, 0, 2, 5, 3, 1, 9, 10, 5],
  [7, 5, 2, 0, 7, 1, 5, 3, 2, 4],
  [9, 5, 5, 7, 0, 7, 3, 5, 5, 3],
  [2, 3, 3, 1, 7, 0, 10, 4, 1, 6],
  [7, 5, 1, 5, 3, 10, 0, 3, 2, 4],
  [1, 6, 9, 3, 5, 4, 3, 0, 4, 4],
  [5, 10, 10, 2, 5, 1, 2, 4, 0, 8],
  [8, 1, 5, 4, 3, 6, 4, 4, 8, 0]],
 [[0, 10, 9, 8, 5, 1, 3, 7, 10, 9, 3, 8, 5, 5, 5, 10, 8, 5, 6, 4],
  [10, 0, 3, 2, 8, 9, 10, 1, 8, 1, 7, 2, 9, 8, 4, 3, 8, 3, 1, 10],
  [9, 3, 0, 5, 8, 6, 9, 1, 4, 1, 9, 5, 5, 8, 7, 2, 2, 6, 3, 1],
  [8, 2, 5, 0, 2, 7, 9, 2, 2, 1, 6, 5, 2, 7, 8, 2, 1, 3, 7, 3],
  [5, 8, 8, 2, 0, 1, 8, 8, 2, 3, 1, 1, 9, 3, 2, 10, 3, 4, 4, 6],
  [1, 9, 6, 7, 1, 0, 9, 1, 1, 1, 6, 1, 1, 9, 5, 4, 6, 4, 8, 6],
  [3, 10, 9, 9, 8, 9, 0, 7, 8, 10, 9, 8, 10, 5, 9, 6, 6, 4, 8, 2],
  [7, 1, 1, 2, 8, 1, 7, 0, 4, 6, 9, 4, 8, 7, 9, 5, 6, 10, 5, 10],
  [10, 8, 4, 2, 2, 1, 8, 4, 0, 3, 10, 5, 4, 2, 10, 7, 8, 8, 6, 6],
  [9, 1, 1, 1, 3, 1, 10, 6, 3, 0, 7, 7, 8, 8, 2, 5, 9, 5, 7, 2

# Teste com o o Algoritmo Prim

In [14]:
qtd_execucao = 20
dados = []

for matriz in grafos_gerados:
    tempo_total = 0
    memoria_total = 0
    
    for _ in range(qtd_execucao):
        resultado, tempo, memoria = measure_performance(prim, (matriz,))
        
        tempo_total += tempo
        memoria_total += memoria
        
    tempo = tempo_total / qtd_execucao
    memoria = memoria_total / qtd_execucao
    
    dados.append((len(matriz), tempo, memoria))
    
    print(f"Tempo: {tempo:.6f} segundos, Memória: {memoria:.2f} KB")
    
    # salvar_grafo_agm_pronto(matriz,f'resultados/{len(matriz)}_vertices.png')

Tempo: 0.000150 segundos, Memória: 5.78 KB
Tempo: 0.000350 segundos, Memória: 24.24 KB
Tempo: 0.000956 segundos, Memória: 55.57 KB
Tempo: 0.001553 segundos, Memória: 100.68 KB
Tempo: 0.002689 segundos, Memória: 156.83 KB
Tempo: 0.003641 segundos, Memória: 227.32 KB
Tempo: 0.005151 segundos, Memória: 310.11 KB
Tempo: 0.006403 segundos, Memória: 408.60 KB
Tempo: 0.008218 segundos, Memória: 523.32 KB
Tempo: 0.010744 segundos, Memória: 645.10 KB
Tempo: 0.012973 segundos, Memória: 773.13 KB
Tempo: 0.016552 segundos, Memória: 919.95 KB
Tempo: 0.019867 segundos, Memória: 1080.02 KB
Tempo: 0.023250 segundos, Memória: 1249.66 KB
Tempo: 0.029349 segundos, Memória: 1434.08 KB


# Plotando Tabelas e Gráficos

In [15]:
dados

[(10, 0.00014995336532592775, 5.776953125),
 (20, 0.0003499150276184082, 24.243359375),
 (30, 0.0009563684463500977, 55.5671875),
 (40, 0.001553356647491455, 100.6765625),
 (50, 0.0026887178421020506, 156.834765625),
 (60, 0.0036408662796020507, 227.322265625),
 (70, 0.005151224136352539, 310.108203125),
 (80, 0.006402814388275146, 408.595361328125),
 (90, 0.008218014240264892, 523.321923828125),
 (100, 0.010744261741638183, 645.0984375),
 (110, 0.012972867488861084, 773.130126953125),
 (120, 0.01655200719833374, 919.951123046875),
 (130, 0.01986660957336426, 1080.024658203125),
 (140, 0.02324965000152588, 1249.65546875),
 (150, 0.029349231719970705, 1434.078466796875)]

In [17]:
dados_df = pd.DataFrame(dados, columns=['Vértices', 'Tempo (s)', 'Memória (KB)'])
dados_df

,Vértices,Tempo (s),Memória (KB)
0,10,0.000150,5.776953
1,20,0.000350,24.243359
2,30,0.000956,55.567188
3,40,0.001553,100.676563
4,50,0.002689,156.834766
5,60,0.003641,227.322266
6,70,0.005151,310.108203
7,80,0.006403,408.595361
8,90,0.008218,523.321924
9,100,0.010744,645.098438


## Gráficos de linha

In [19]:
plt.figure(figsize=(10, 6))
plt.plot(dados_df['Vértices'], dados_df['Tempo (s)'], marker='o', label='Tempo (s)')
plt.plot(dados_df['Vértices'], dados_df['Memória (KB)'], marker='s', label='Memória (KB)')
plt.xlabel('Quantidade de Vértices')
plt.ylabel('Tempo (s) / Memória (KB)')
plt.title('Desempenho do Algoritmo de Prim')
plt.legend()
plt.grid(True)
plt.show()